In [1]:
!git clone https://github.com/sosamandara/token_reduction_nlp.git

Cloning into 'token_reduction_nlp'...
remote: Enumerating objects: 354, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (303/303), done.
remote: Total 354 (delta 81), reused 314 (delta 46), pack-reused 0
Receiving objects: 100% (354/354), 15.12 MiB | 10.74 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [2]:
%cd /content/token_reduction_nlp/notebooks

/content/token_reduction_nlp/notebooks


In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Assuming the notebooks are in the notebooks directory and executed from there
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Add the models and src directories to the Python path
models_dir = os.path.join(project_root, 'models')
src_dir = os.path.join(project_root, 'src')

sys.path.append(models_dir)
sys.path.append(src_dir)

# Verify that the paths are correctly added
print("Current directory:", current_dir)
print("Project root directory:", project_root)
print("Models directory added to sys.path:", models_dir in sys.path)
print("Src directory added to sys.path:", src_dir in sys.path)

Current directory: d:\TESI\token_reduction_nlp\notebooks
Project root directory: d:\TESI\token_reduction_nlp
Models directory added to sys.path: True
Src directory added to sys.path: True


In [2]:
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AutoTokenizer, AutoModel
from modeling_topK_gpt2 import CustomGPT2LMHeadModel
import json
import matplotlib.pyplot as plt
from functions import run_generation_on_dataset, plot_averages, load_custom_model
import numpy as np
import pandas as pd

d:\Conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
d:\Conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


# some trial

In [3]:
def calculate_reduction(lenght, mask):
  actual_lenght=lenght
  for percentage in mask:
    actual_lenght = (actual_lenght - int(actual_lenght*percentage))
  return actual_lenght + 1

window_sizes = [71, 50, 100, 200, 300, 500, 1000]
remaining_tokens = []
per_keep = []

mask_90 = [0.0, 0.0, 0.0, 0.0, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_80 = [0.0, 0.0, 0.0, 0.0, 0.22, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_70 = [0.0, 0.0, 0.0, 0.0, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_60 = [0.0, 0.0, 0.0, 0.0, 0.42, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_50 = [0.0, 0.0, 0.0, 0.0, 0.52, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_40 = [0.0, 0.0, 0.0, 0.0, 0.62, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_30 = [0.0, 0.0, 0.0, 0.0, 0.72, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mask_20 = [0.0, 0.0, 0.0, 0.0, 0.82, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
masks = [mask_20, mask_30, mask_40, mask_50,
         mask_60, mask_70, mask_80, mask_90,]
mask_number = ["16", "30", "40", "50", "60", "70", "80", "90"]
for mask in masks:
  per_keep = []
  for w in window_sizes:
    #remaining_tokens.append(calculate_reduction(w, mask))
    per_keep.append(round(calculate_reduction(w, mask)/w,2))
    #print(remaining_tokens)
  print("----------")
  print(per_keep)

----------
[0.2, 0.2, 0.19, 0.18, 0.19, 0.18, 0.18]
----------
[0.3, 0.3, 0.29, 0.28, 0.28, 0.28, 0.28]
----------
[0.39, 0.4, 0.39, 0.39, 0.38, 0.38, 0.38]
----------
[0.51, 0.5, 0.49, 0.48, 0.48, 0.48, 0.48]
----------
[0.61, 0.6, 0.59, 0.58, 0.58, 0.58, 0.58]
----------
[0.7, 0.7, 0.69, 0.69, 0.68, 0.68, 0.68]
----------
[0.8, 0.8, 0.79, 0.79, 0.78, 0.78, 0.78]
----------
[0.9, 0.9, 0.89, 0.89, 0.88, 0.88, 0.88]


In [4]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
config = GPT2Config.from_pretrained(model_name)
model_gpt = GPT2LMHeadModel.from_pretrained(model_name, config=config).to('cuda')

# Layers to prune
layers_to_prune = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

#model_id = f"top_K_90"
#top_K_90 =  load_custom_model(model_name, config, mask_90, selection_method="top_k", layers_to_prune=layers_to_prune)

custom_models = {}
for i, mask in enumerate(masks):
    model_id = f"top_k_{i * 10 + 20}"
    custom_models[model_id] = load_custom_model(model_name, config, mask, selection_method="top_k", layers_to_prune=layers_to_prune)

d:\Conda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
# Function to generate text and calculate FLOPs
def generate_text(model, tokenizer, input_text, max_length=100):
    # tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')

    # generate text
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    print(type(model.indices), input_ids[model.indices[0]])
    # decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text, model.indices

In [6]:
generated_text = """The problem is not the problem. The problem is your attitude about the problem."""
for i in range(len(tokenizer.encode(generated_text, return_tensors='pt').to('cuda')[0])):
    print(str(i)+"--------------"+tokenizer.decode(tokenizer.encode(generated_text, return_tensors='pt').to('cuda')[0][i]))

0--------------The
1-------------- problem
2-------------- is
3-------------- not
4-------------- the
5-------------- problem
6--------------.
7-------------- The
8-------------- problem
9-------------- is
10-------------- your
11-------------- attitude
12-------------- about
13-------------- the
14-------------- problem
15--------------.


In [ ]:
The problem is not the problem . The problem is your attitude about the problem .
.,is,problem,the,problem,not,The,is,problem,problem,the
90,80,80,70,70,60,50,50,40,30,30
6,2,5,4,1,3,7,9,14,8,13

In [9]:
indices

tensor([[ 6,  2,  5,  4,  1,  3,  7,  9, 14,  8, 13]], device='cuda:0')

In [ ]:
'.': '90%',          #6
'is': '80%',         #2
'problem': '80%',    #5
'the': '70%',        #4
'problem': '70%',    #1
'not': '60%',        #3
'The': '50',         #7
'is': '50',          #9
'problem': '40',     #14
'problem': '30',     #8
'the': '30',         #13

In [ ]:
generated_text = """The problem is not the problem. The problem is your attitude about the problem."""
generated_text, indices = generate_text(top_K_30, tokenizer, generated_text, max_length=len(tokenizer(generated_text)["input_ids"])+1)

In [8]:
generated_text = """Life is like a box of chocolates, you never know what you're gonna get."""
for i in range(len(tokenizer.encode(generated_text, return_tensors='pt').to('cuda')[0])):
    print(str(i)+"--------------"+tokenizer.decode(tokenizer.encode(generated_text, return_tensors='pt').to('cuda')[0][i]))

0--------------Life
1-------------- is
2-------------- like
3-------------- a
4-------------- box
5-------------- of
6-------------- ch
7--------------ocol
8--------------ates
9--------------,
10-------------- you
11-------------- never
12-------------- know
13-------------- what
14-------------- you
15--------------'re
16-------------- gonna
17-------------- get
18--------------.


In [16]:
generated_text = """Life is like a box of chocolates, you never know what you're gonna get."""
generated_text = generate_text(custom_models["top_k_30"], tokenizer, generated_text, max_length=len(tokenizer(generated_text)["input_ids"])+1)

torch.Size([1, 12, 19, 19])
torch.Size([1, 12, 19, 19])
torch.Size([1, 12, 19, 19])
torch.Size([1, 12, 19, 19])
torch.Size([1, 12, 19, 19])
tensor([[ 1,  3,  9,  5,  6,  4, 10,  8,  2,  7, 12, 16, 11]], device='cuda:0') torch.Size([1, 19])
torch.Size([1, 12, 6, 6])
torch.Size([1, 12, 6, 6])
torch.Size([1, 12, 6, 6])
torch.Size([1, 12, 6, 6])
torch.Size([1, 12, 6, 6])
torch.Size([1, 12, 6, 6])
torch.Size([1, 12, 6, 6])


In [ ]:
Life is like a box of ch ocol ates  , you never know what you 're gonna get .
is:a:,:of:ch:box:you:ates:like:ocol:know:never:gonna
90,90,80,80,70,70,60,50,50,40,40,30,30
1,3,9,5,6,4,10,8,2,7,12,11,16

In [ ]:
'is': '90%',         #1
'a': '90%',          #3
',': '80%',          #9
'of': '80%',         #5
'ch': '70%',         #6
'box': '70%',        #4
'you': '60',         #10
'ates': '50',        #8
'like': '50',        #2
'ocol': '40',        #7
'know': '40',        #12
'gonna': '30',        #16
'never': '30',        #11

In [ ]:
generated_text = generate_text(top_K_90, tokenizer, generated_text, max_length=len(tokenizer(generated_text)["input_ids"])+1)

torch.Size([1, 12, 24, 24])
torch.Size([1, 12, 24, 24])
torch.Size([1, 12, 24, 24])
torch.Size([1, 12, 24, 24])
torch.Size([1, 12, 24, 24])
tensor([[7, 5]], device='cuda:0') torch.Size([1, 24])
torch.Size([1, 12, 22, 22])
torch.Size([1, 12, 22, 22])
torch.Size([1, 12, 22, 22])
torch.Size([1, 12, 22, 22])
torch.Size([1, 12, 22, 22])
torch.Size([1, 12, 22, 22])
torch.Size([1, 12, 22, 22])
